<a href="https://colab.research.google.com/github/so-dipe/so-dipe/blob/main/DeepSynergy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install category_encoders

     |████████████████████████████████| 82 kB 312 kB/s 


In [2]:
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from category_encoders import OneHotEncoder

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


First, Import the Data.

In [3]:
df = pd.read_csv('/content/drive/MyDrive/Dataset.csv')
df.head()

,BatchID,cell_line,drugA_name,drugA Conc (µM),drugB_name,drugB Conc (µM),combination_name,viability1,viability2,viability3,viability4,mu/muMax,X/X0
0,1,A2058,5-FU,0.35,ABT-888,0.35,5-FU & ABT-888,0.971,1.090,0.949,0.996,0.992,0.988
1,1,A2058,5-FU,0.35,ABT-888,1.08,5-FU & ABT-888,1.124,0.180,0.917,0.962,0.968,0.957
2,1,A2058,5-FU,0.35,ABT-888,3.25,5-FU & ABT-888,1.043,0.878,0.922,0.910,0.955,0.940
3,1,A2058,5-FU,0.35,ABT-888,10.00,5-FU & ABT-888,0.919,0.784,0.795,0.788,0.882,0.849
4,1,A2058,5-FU,1.08,ABT-888,0.35,5-FU & ABT-888,1.029,0.939,0.879,0.992,0.982,0.976


From the above, we see some useful information about the drugs, the cell line and the drug synergy.
Drug Synergy is represented by the X/X0 row, which is a normalized version of mu/muMax. 
The values are between 0 and 1, where 0 represents inhibition of cell growth and 1 represnts no effect.

In [4]:
df['cell_line'].nunique()

39

From the abve, there are 39 different cell line in the dataset. Using an categorical encoder, we create 39 new columns for each cell line 

In [5]:
Le = OneHotEncoder()
Le.fit(df['cell_line'])
df[df['cell_line'].unique()] = Le.transform(df['cell_line'])
df.head()

,BatchID,cell_line,drugA_name,drugA Conc (µM),drugB_name,drugB Conc (µM),combination_name,viability1,viability2,viability3,...,SKOV3,SW620,SW837,T47D,UACC62,UWB1289,UWB1289BRCA1,VCAP,ZR751,MSTO
0,1,A2058,5-FU,0.35,ABT-888,0.35,5-FU & ABT-888,0.971,1.090,0.949,...,0,0,0,0,0,0,0,0,0,0
1,1,A2058,5-FU,0.35,ABT-888,1.08,5-FU & ABT-888,1.124,0.180,0.917,...,0,0,0,0,0,0,0,0,0,0
2,1,A2058,5-FU,0.35,ABT-888,3.25,5-FU & ABT-888,1.043,0.878,0.922,...,0,0,0,0,0,0,0,0,0,0
3,1,A2058,5-FU,0.35,ABT-888,10.00,5-FU & ABT-888,0.919,0.784,0.795,...,0,0,0,0,0,0,0,0,0,0
4,1,A2058,5-FU,1.08,ABT-888,0.35,5-FU & ABT-888,1.029,0.939,0.879,...,0,0,0,0,0,0,0,0,0,0


Now, to drop some of the columns that aren't needed...

In [6]:
df.drop(columns=['BatchID', 'cell_line', 'combination_name', 'viability1', 'viability2', 'viability3', 'viability4'], inplace=True)
df.head()

,drugA_name,drugA Conc (µM),drugB_name,drugB Conc (µM),mu/muMax,X/X0,A2058,A2780,A375,A427,...,SKOV3,SW620,SW837,T47D,UACC62,UWB1289,UWB1289BRCA1,VCAP,ZR751,MSTO
0,5-FU,0.35,ABT-888,0.35,0.992,0.988,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,5-FU,0.35,ABT-888,1.08,0.968,0.957,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,5-FU,0.35,ABT-888,3.25,0.955,0.940,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,5-FU,0.35,ABT-888,10.00,0.882,0.849,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5-FU,1.08,ABT-888,0.35,0.982,0.976,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


To create the DeepSynergy Model, we use to deep learning formats.

1. Rectanglar, where all the hidden layers have same number of nodes
2. Conic, where the one hidden layer is half the next as we move forward in the network from Input to Output

The output of the model should be a number from 0 to 1.

In [7]:
def rect(num_cell_line=39, layers=2, neurons=2048):
    model = Sequential()
    model.add(Dense(neurons ,input_shape=(4 + num_cell_line, ), activation='relu'))
    for i in range(layers):
        model.add(Dense(neurons, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.summary()
    return model

rect()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 2048)              90112     
                                                                 
 dense_1 (Dense)             (None, 2048)              4196352   
                                                                 
 dense_2 (Dense)             (None, 2048)              4196352   
                                                                 
 dense_3 (Dense)             (None, 1)                 2049      
                                                                 
Total params: 8,484,865
Trainable params: 8,484,865
Non-trainable params: 0
_________________________________________________________________


Note that from the above, input_shape is 4 + num_cell_line, where num_cell_line is the number of cell line present in the dataset, the default being 39. The 4 represents the chemical information and genomic information of both drugs

In [8]:
def conic(num_cell_line=39, layers=2, first_neurons=2048):
    model = Sequential()
    model.add(Dense(first_neurons ,input_shape=(4 + num_cell_line, ), activation='relu'))
    for i in range(layers):
        first_neurons = first_neurons/2
        model.add(Dense(first_neurons, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.summary()
    return model


conic()   

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 2048)              90112     
                                                                 
 dense_5 (Dense)             (None, 1024)              2098176   
                                                                 
 dense_6 (Dense)             (None, 512)               524800    
                                                                 
 dense_7 (Dense)             (None, 1)                 513       
                                                                 
Total params: 2,713,601
Trainable params: 2,713,601
Non-trainable params: 0
_________________________________________________________________
